<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Full_Test_Jacobians_Resnets_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for generating Full Jacobian SVD over the test data for ResNet data


June 5 2020

```
resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
```




In [0]:
Jdir =  '/content/drive/My Drive/J_resnets_fullsvd'

thismodel = 'resnet542bn_cifar10'
start_batch = 0

batch_size = 100

num_classes= 10
device = 'cuda:0'
num_data = 10000


In [3]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [4]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=60dd5f9f3482a198191af29daeb7f25d756f39a7a14a1a091bb31644ad410aa4
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 43.2 s


In [9]:
!mkdir "$Jdir"
!ls "$Jdir"

mkdir: cannot create directory ‘/content/drive/My Drive/J_resnets_fullsvd’: File exists
data  Jfull_resnet1001_cifar10__0.csv
time: 1.45 s


In [10]:
!pip install pytorchcv
!pip install powerlaw
!pip install GPUtil
import GPUtil

time: 8.26 s


In [11]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

import gc


/bin/bash: import: command not found
time: 4.27 s


### Run InstallPhysTheory notebook 

In [12]:
import sys
sys.path.append("/content/drive/My Drive/Jacobian")
import jacobian as jac
import utils

time: 543 ms


In [13]:
def jacobian_batched_full(model, data_loader, batch_size, start_batch = 0, num_classes=10, device='cuda:0', data_dim=3*32*32, formJJ=False, save=True): 
  '''Compute a full of J^{T}J Jacobian in batch mode.  '''

  model.eval() 
  model = model.to(device)

  left_data_loader = data_loader 
  right_data_loader = copy.deepcopy(data_loader)

  JJMat = None 
  for left_batch, data in enumerate(left_data_loader): 
    if left_batch < start_batch:
      continue
    print(left_batch) 

    inputs, _ = data 
    inputs = inputs.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)
    J = jac.batch_diagJ(inputs, outputs)
    J = J.reshape(batch_size,num_classes*data_dim)

    X = J.detach().cpu().numpy()

    del data, inputs, outputs, J
    gc.collect()
    torch.cuda.empty_cache()
    GPUtil.showUtilization()

    JJBlock = None
    for right_batch, data in enumerate(right_data_loader):
      print(left_batch, right_batch)
      inputs, _ = data 
      inputs = inputs.to(device)
      inputs.requires_grad=True
      outputs = model(inputs)
    
      Jt = jac.batch_diagJ(inputs, outputs)
      Jt = Jt.reshape(batch_size,num_classes*data_dim)

      Xt = Jt.detach().cpu().numpy().transpose()

      del data, inputs, outputs, Jt
      gc.collect()
      torch.cuda.empty_cache()

      block = np.dot(X,Xt)
      JJBlock = np.hstack([JJBlock, block]) if JJBlock is not None else block

    # end of right_batch loop
    
    if save:
      filename = "{}/Jfull_{}__{}.csv".format(Jdir, modelname, left_batch)
      np.savetxt(filename, JJBlock, delimiter='\t')
      print("Saved file ",filename)

    if formJJ:
      JJMat = np.vstack([JJMat, JJBlock]) if JJMat is not None else JJBlock

      print("JJMat shape ", left_batch, JJMat.shape, filename) 

  # end of left_batch loop

  return JJMat


time: 47.4 ms


In [14]:
def jacobian_full(model, data_loader, batch_size, num_classes=10, device='cuda:0', data_dim=3*32*32): 
  '''compute a full of J^{T}J Jacobian, as a torch tensor'''

  model.eval()
  model = model.to(device)

  J_right = jac.construct_diagJ(model, test_loader, batch_size=batch_size, device=device) 
  J_left = J_right.transpose(1,0)
  JJ = torch.mm(J_right, J_left)

  del J_right, J_left
  gc.collect()
  torch.cuda.empty_cache()

  return JJ

time: 5.9 ms


### ResNet 

In [15]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

for modelname in pytorchcv.model_provider._models.keys():
    if modelname.startswith('resnet') and modelname.endswith('cifar10'):
        print(modelname)

resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
time: 110 ms


### Compute JJ, in batches or all at once


In [17]:
print(thismodel)
train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

model = ptcv_get_model(thismodel, pretrained=True)  
jacobian_batched_full(model, test_loader, batch_size=batch_size, start_batch=start_batch)


resnet542bn_cifar10


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified

0
| ID | GPU | MEM |
------------------
|  0 | 88% |  5% |
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15


KeyboardInterrupt: ignored

time: 1min 33s


50.0

time: 2.28 ms
